In [1]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=43d5b726b8118cf6d27f1c18ddb02dc151c8d2a52f5a9a00b616f40279dbf1c1
  Stored in directory: /aiffel/.cache/pip/wheels/60/b3/27/d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import json
import torch
import random
import sacrebleu
import numpy as np
import pandas as pd
import tensorflow as tf
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import warnings

In [3]:
import googletrans

In [4]:
MBart_tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/649 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/529 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

In [5]:
MBart_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [6]:
translator = googletrans.Translator()

In [7]:
def data_import(data_name):
    data_path = os.getenv('HOME')+'/aiffel/project_data/dlthon/'+data_name
    imported_data = pd.read_csv(data_path)
    return imported_data

In [20]:
def sigle_translate(input_data, s_lang, t_lang, top_k_val, type_trans = 'forward'):
    original_list = []
    return_list = []
    class_list = []
    MBart_tokenizer.src_lang = s_lang
    if type_trans == 'forward':
        single_data = input_data['conversation']
        for i,row_input_data in enumerate(single_data):
            retrun_text_list = []
            text_sen = row_input_data
            text_list = text_sen.split('\n')
            for text in text_list:
                encoded_text = MBart_tokenizer(text, return_tensors="pt")
                generated_tokens = MBart_model.generate(**encoded_text, forced_bos_token_id = MBart_tokenizer.lang_code_to_id [t_lang])
                translation = MBart_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                retrun_text_list.extend(translation)
            retrun_text = '\n'.join(retrun_text_list)
            class_list.append(input_data['class'][0])
            original_list.append(text_sen)
            return_list.append(retrun_text)
            print(i)
    else:
        for i, text_sen in enumerate(input_data['conversation']):
            retrun_text_list = []
            text_list = text_sen.split('\n')
            for text in text_list:
                encoded_text = MBart_tokenizer(text, return_tensors="pt")
                generated_tokens = MBart_model.generate(**encoded_text, forced_bos_token_id = MBart_tokenizer.lang_code_to_id [t_lang], do_sample=True, top_k=top_k_val)
                translation = MBart_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                retrun_text_list.extend(translation)
            retrun_text = '\n'.join(retrun_text_list)
            class_list.append(input_data['class'][0])
            original_list.append(text_sen)
            return_list.append(retrun_text)
            print(i)
    
    original_data = pd.DataFrame({'class' : class_list, 'conversation' : original_list})
    translated_data = pd.DataFrame({'class' : class_list, 'conversation' : return_list})
        
    return original_data, translated_data

In [27]:
def sigle_translate_google(input_data, s_lang, t_lang, type_trans = 'forward'):
    original_list = []
    return_list = []
    class_list = []
    if type_trans == 'forward':
        single_data = input_data['conversation']
        for i, row_input_data in enumerate(single_data):
            retrun_text_list = []
            text_sen = row_input_data
            text_list = text_sen.split('\n')
            for text in text_list:
                translation = translator.translate(text, src=s_lang, dest=t_lang).text
                retrun_text_list.append(translation)
            retrun_text = '\n'.join(retrun_text_list)
            class_list.append(input_data['class'][0])
            original_list.append(text_sen)
            return_list.append(retrun_text)
            print(i)
    else:
        for i, text_sen in enumerate(input_data['conversation']):
            retrun_text_list = []
            text_list = text_sen.split('\n')
            for text in text_list:
                translation = translator.translate(text, src=s_lang, dest=t_lang).text
                retrun_text_list.append(translation)
            retrun_text = '\n'.join(retrun_text_list)
            class_list.append(input_data['class'][0])
            original_list.append(text_sen)
            return_list.append(retrun_text)
            print(i)
        
    original_data = pd.DataFrame({'class' : class_list, 'conversation' : original_list})
    translated_data = pd.DataFrame({'class' : class_list, 'conversation' : return_list})
        
    return original_data, translated_data

In [10]:
def back_translate(input_data, s_lang, t_lang, top_k_val):
    original_data1, translated_data = sigle_translate(input_data, s_lang, t_lang, 'forward')
    original_data2, return_data = sigle_translate(translated_data, t_lang, s_lang, top_k_val, 'backward')
    return [original_data1, return_data]

In [11]:
def back_translate_google(input_data, s_lang, t_lang):
    original_data1, translated_data = sigle_translate_google(input_data, s_lang, t_lang, 'forward')
    original_data2, return_data = sigle_translate_google(translated_data, t_lang, s_lang, 'backward')
    return [original_data1, return_data]

In [12]:
def back_translate_colabo(input_data, s_lang_g, t_lang_g, s_lang, t_lang, top_k_val):
    original_data1, translated_data = sigle_translate_google(input_data, s_lang_g, t_lang_g, 'forward')
    original_data2, return_data = sigle_translate(translated_data, t_lang, s_lang, top_k_val, 'backward')
    return [original_data1, return_data]

In [13]:
def eval_bule(back_translate_data):
    return_score = []
    hypothesis = back_translate_data[1]['conversation']
    references = back_translate_data[0]['conversation']
    for hyp, ref in zip(hypothesis, references):
        hypothese = [hyp]
        reference = [ref]
        bleu = sacrebleu.corpus_bleu(hypothese, [reference])
        return_score.append(bleu.score)
    mean_score = np.mean(return_score)
    return mean_score

In [14]:
train_data = data_import('train.csv')

train_data = train_data.drop(train_data.columns[0], axis=1)
threat_data = train_data[train_data['class'] == '협박 대화']
extort_data = train_data[train_data['class'] == '갈취 대화']
co_bully_data = train_data[train_data['class'] == '직장 내 괴롭힘 대화']
bully_data = train_data[train_data['class'] == '기타 괴롭힘 대화']

threat_data.reset_index(drop=True, inplace=True)
extort_data.reset_index(drop=True, inplace=True)
co_bully_data.reset_index(drop=True, inplace=True)
bully_data.reset_index(drop=True, inplace=True)

threat_data = threat_data.astype(str)
extort_data = extort_data.astype(str)
co_bully_data = co_bully_data.astype(str)
bully_data = bully_data.astype(str)

In [15]:
len(threat_data)

896

In [16]:
threat_input_data = threat_data.sample(448).reset_index(drop=True)

In [17]:
threat_input_data_list = []
k = 0

for i in range(10, 451, 10):
    threat_inputs = threat_input_data.loc[k:i]
    threat_input_data_list.append(threat_inputs)
    k+=10
    
len(threat_input_data_list)

45

In [21]:
threat_input_data_list[0]

,class,conversation
0,협박 대화,너 너무 맘에 안들어 진짜 죽이고 싶어\n같이 오늘 한번 아작을 내볼까?\n미안해\...
1,협박 대화,안녕하세요\n 네? 네 안녕하세요\n 혹시 혈액형이 어떻게 되세요?\n 네? 혈액형...
2,협박 대화,내가 맘만 먹으면 너같은 거 여기 발도 못붙이게 할 수있는거 알지?\n제가 뭘 잘못...
3,협박 대화,너지?\n뭐가?\n내가 전직장에서 불미스러운 일에 엮여서 회사 여기로 옮긴거말이야\...
4,협박 대화,내 손에 뭐가 있는것같아?\n에이 장난치지마세요\n너 눈엔 이게 가짜 총으로 보이니...
5,협박 대화,니네가족 한참 좋을때지?\n죄송합니다. \n죄송?이라는말 필요없고 내가 전에 얘기한...
6,협박 대화,왜 그러세요 아저씨\n여자 새끼들은 한결같이 얌체 같아 넌 살 자격이 없어\n무슨 ...
7,협박 대화,빨리 돈 구해와.\n저 정말 돈 나올데가 없어요. 제발요.\n니네 부모님 가게 정리...
8,협박 대화,머해? 나 그것 좀 주라\n응? 안돼. 이거 생인선물로 받은거야.\n야 어차피 몰라...
9,협박 대화,어제까지 완료 하기로 했던 일은 다 완료 되었나요?\n어제 급한 일이 있어서 처리를...


In [23]:
threat_bt_colab0 = back_translate_colabo(threat_input_data_list[0], 'ko', 'en', 'ko_KR', 'en_XX', 3)

0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10


In [31]:
threat_bt_google0 = back_translate_google(threat_input_data_list[0], 'ko', 'en')

0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10


In [29]:
threat_bt0 = back_translate(threat_input_data_list[0], 'ko_KR', 'en_XX', 3)

0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10


In [25]:
eval_bule(threat_bt_colab0)

2.9631089451969896

In [32]:
eval_bule(threat_bt_google0)

3.8329663759924886

In [30]:
eval_bule(threat_bt0)

2.2771710719511318